# Summaries and Tensorboard
In this notebook we extend the previous example by adding summaries to the computation graph, 
and writing these summaries to a logfile in the training loop. These logfiles can than be looked at in tensorboard, a utility program that comes with tensorflow. It is tremendously helpful when debugging a model, or even for simply monitoring its progress.

To get a more interesting learning graph the number of training steps has been increased.

Summary operations take a tensor value and serialize it to a string that can be used for logging/monitoring. 
Summary operations are part of `tf.summary` and include
* `tf.summary.scalar`
* `tf.summary.image`
* `tf.summary.audio`
* `tf.summary.histogram`
* `tf.summary.text`

Multiple summaries an be joined using `tf.summary.merge`.

In [ ]:
import tensorflow as tf
import tensorflow.contrib.learn as tflearn
import numpy as np

NUM_STEPS = 500

## Building the Network
This code is mostly as before, but with a few summary operations sprinkled in. 
We generate a histogram of the logit and probability values. 

One thing that is always recommended to do is to monitor your input data in the network. 
This is to make sure that the data after preprocessing is actually something the network can work with.
(e.g. the image format is as expected, cropping and resizing do not destroy the information we want to learn from etc.)

To visualize the image we need to convert the column vector into an image shaped tensor. The last dimension gets a size of 1, indicating a grayscale image. The batch dimension gets a size of -1 to indicate that the value is to be chosen such that the total number of elements matches up. (We cannot pass `None` here, since the new shape is a Tensor, and as such each entry needs to be of type int). 

In [ ]:
x = tf.placeholder(tf.float32, (None, 784), name="x")
tf.summary.image("image", tf.reshape(x, (-1, 28, 28, 1)))

y = tf.placeholder(tf.int32, (None), name="y")

W = tf.Variable(np.random.random((784, 10)), dtype=tf.float32, name="W")
b = tf.Variable(np.random.random(10), dtype=tf.float32, name="b")

l = tf.matmul(x, W) + b
tf.summary.histogram("logits", l)

p = tf.nn.softmax(l)
tf.summary.histogram("probabilities", p)

## Loss and Optimizer
This is almost the same as before. We moved the loss calculation into a `name_scope` to get the graph a bit more structured, and added a summary for the `loss`.

In [ ]:
with tf.name_scope("loss_calculation"):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=tf.one_hot(y, depth=10), logits=l)
    loss = tf.reduce_mean(loss)
tf.summary.scalar("loss", loss)

optimizer = tf.train.GradientDescentOptimizer(0.1)
train_op = optimizer.minimize(loss)

We now calculate the accuracy of our predictions. 
We put all operations into a name scope to get a more readable graph. The `name_scope` function takes an optional second argument which is a list of tensors we want to use within this scope. This checks that all Tensors are defined in the same graph, but can usually be omitted when writing a model (everything is in the default graph anyway).

The calculation of the accuracy is done like this: First we calculate the prediced class (as an integer), and compare that to the given labels. This results in a boolean tensor, so before we calculate the mean we cast it to a floating point type.

In [ ]:
with tf.name_scope("accuracy_calc", [l, y]):
    predicted = tf.argmax(l, axis=1, name="predicted")
    correct = tf.equal(tf.cast(predicted, tf.int32), y, name="is_correct")
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32), name="accuracy")
tf.summary.scalar("accuracy", accuracy)

## Loading Data and Initialization
This remains unchanged from the previous example.

In [ ]:
# load the dataset and create a session
mnist = tflearn.datasets.load_dataset("mnist")
images = mnist.train.images
labels = mnist.train.labels

session = tf.InteractiveSession()

# Important: Initialize the Variables.
tf.global_variables_initializer().run()

## Summary Writer
Here we create the summary writer, that writes all summaries into the `tensorboard_demo` directory. It immediately write a description of the computation graph associated with `session`, which is just the default graph in which we built our model. We also create a *merged* summary operation that combiens all the summaries defined above.

Please note that by using a fixed path name, executing this notebook for the second time will write another log file into the same directory, which will lead to artifacts in tensorboard visualizations. If you execpt to train the model multiple times, either delete the summary directory before restarting, or write into subdirectory for different runs. The last option is also necessary if you want to compare different runs in tensorboard.

In [ ]:
summaries = tf.summary.merge_all()
writer = tf.summary.FileWriter("tensorboard_demo", session.graph)

## Training Loop
Instead of the loss scalar, we now extract the merge summary. The result is added to the `writer`. The `add_summary` function also expects a time step, so we put in the loop counter. This is used for the x axis in the tensorboard plots. Finally, we close the `writer` again. This ensures that all summaries that have not yet been written to file will be written now.

In [ ]:
for i in range(NUM_STEPS):
    summary, _ = session.run([summaries, train_op], {x: images, y: labels})
    writer.add_summary(summary, i)

writer.close()

## Tensorboard
To view the logs in tensorboard you can simply start the tensorboard server using
```bash
>> tensorboard --logdir tensorboard_demo
```
and then visit `127.0.0.1:6006` with your webbrowser. If that port is in use for some reason, you can specify the `--port` option when starting tensorboard. (On some machines `127.0.0.1` does not seem to work, you can try `localhost` instead). 

If you run your tensorflow code on another machine (e.g. on the cluster) and want to folow along with tensorboard, there are three options:
- You may be able to simply access the tensorboard address on the remote system by going to `[REMOTE_ADDRESS]:6006`.
- Set the logdir to a network mounted filesystem and run tensorboard locally, pointing to the network files.
- Use an ssh redirection to mirror the address to your local system 
```bash
>>> ssh -L 6006:127.0.0.1:6006 [USER]@[REMOTE_ADDRESS]
```